# DiffEdit

In [ ]:
import argparse
import torch
import os
import numpy as np
from omegaconf import OmegaConf
from pytorch_lightning import seed_everything
from torch import autocast
from contextlib import nullcontext
import random
from PIL import Image
from einops import repeat
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

from ldm.util import instantiate_from_config
from ldm.models.diffusion.dpm_solver import DPMSolverSampler
from ldm.models.diffusion.dpm_solver import (
    model_wrapper,
    NoiseScheduleVP,
    DPM_Solver
)


def load_model_from_config(config, ckpt, verbose=False):
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt, map_location="cpu")
    if "global_step" in pl_sd:
        print(f"Global Step: {pl_sd['global_step']}")
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    if len(m) > 0 and verbose:
        print("missing keys:")
        print(m)
    if len(u) > 0 and verbose:
        print("unexpected keys:")
        print(u)

    model.eval()
    return model

def load_img(path, opt):
    image = Image.open(path).convert("RGB")
    #     w, h = image.size
    #     print(f"loaded input image of size ({w}, {h}) from {path}")
    #     w, h = map(lambda x: x - x % 32, (w, h))  # resize to integer multiple of 32
    image = image.resize((opt.W, opt.H), resample=Image.LANCZOS)
    image = np.array(image).astype(np.float32) / 255.0
    image = image[None].transpose(0, 3, 1, 2)
    image = torch.from_numpy(image)
    return 2. * image - 1.


def latent_to_image(model, latents):
    x_samples = model.decode_first_stage(latents)
    x_samples = torch.clamp((x_samples + 1.0) / 2.0, min=0.0, max=1.0)
    x_samples = x_samples.cpu().permute(0, 2, 3, 1).numpy()
    x_samples = 255. * x_samples
    x_samples = x_samples.astype(np.uint8)

    return x_samples


def repeat_tensor(x, n, dim=0):
    dims = len(x.shape) * [1]
    dims[dim] = n
    return x.repeat(dims)


def get_mask(model, src, dst, init_latent, n: int, ddim_steps,
             clamp_rate:float=3):
    """
    the map value will be clamped to map.mean() * clamp_rate, then values will be scaled into 0~1, then term into binary(split at 0.5).
    so if a map value is large than map.mean() * clamp_rate * 0.5 will be encode to 1, less will be encode to 0. 
    so the larger clamp rate is, less pixes will be encode to 1, the small clamp rate is, the more pixes will be encode to 1.
    """
    device = model.device
    repeated = repeat_tensor(init_latent, n)
    src = repeat_tensor(src, n)
    dst = repeat_tensor(dst, n)


    scheduler = DDIMScheduler(num_train_timesteps=model.num_timesteps,
                              trained_betas=model.betas.cpu().numpy())
    scheduler.set_timesteps(ddim_steps, device=device)
    noised = scheduler.add_noise(repeated, noise,
                                 scheduler.timesteps[ddim_steps // 2]
                                 )

    t = scheduler.timesteps[ddim_steps // 2]
    t_ = torch.unsqueeze(t, dim=0).to(device)
    pre_src = model.apply_model(noised, t_, src)
    pre_dst = model.apply_model(noised, t_, dst)

    # consider to add smooth method
    subed = (pre_src - pre_dst).abs_().mean(dim=[0, 1])
    max_v = subed.mean() * clamp_rate
    mask = subed.clamp(0, max_v) / max_v

    def to_binary(pix):
        if pix > 0.5:
            return 1.
        else:
            return 0.

    mask = mask.cpu().apply_(to_binary).to(device)

    return mask




def diffedit(model, init_image,
             src_prompt: str = "A bowl of fruits",
             dst_prompt: str = "A bowl of pears",
             encode_ratio: float = 0.6,
             ddim_steps: int = 20,
             seed: int = 42,
             scale: float = 7.5,
             precision="autocast"):
    """
    :param init_image: image to be edit
    :param src_prompt: prompt describe origin image(i.e. A bowl of fruits)
    :param dst_prompt: prompt describe desired image(i.e. A bowl of pears)
    :param encode_ratio: how deep to encode origin image, must between 0-1
    :param ddim_steps: total ddim steps, actual encode steps = ddim_steps * encode ratio
    :param seed: random seed
    :param scale: classifier free guidance scale
    :param precision: ema precision
    """
    #If seed is None, randomly select seed from 0 to 2^32-1
    if seed is None:
        seed = random.randrange(2 ** 32 - 1)
    seed_everything(seed)
    device = model.device

    model.cond_stage_model = model.cond_stage_model.to(device)
    precision_scope = autocast if precision == "autocast" else nullcontext
    assert os.path.isfile(opt.origin_image)
    init_image = load_img(opt.origin_image, opt).to(device)
    init_image = repeat(init_image, '1 ... -> b ...', b=1)

    with torch.no_grad():
        with precision_scope(device.type):
            with model.ema_scope():
                uc = None
                if scale != 1.0:
                    uc = model.get_learned_conditioning([""])
                src = model.get_learned_conditioning([src_prompt])
                dst = model.get_learned_conditioning([dst_prompt])
                init_latent = model.get_first_stage_encoding(model.encode_first_stage(init_image))

                # get mask
                mask = get_mask(model, src, dst, init_latent, 3, ddim_steps)
                plt.imshow(mask.detach().cpu().numpy())
                plt.show()
                
                ns = NoiseScheduleVP('discrete', betas=model.betas)
                model_fn = model_wrapper(
                    lambda x, t, c: model.apply_model(x, t, c),
                    ns,
                    model_type="noise",
                    guidance_type="classifier-free",
                    condition=src,
                    unconditional_condition=uc,
                    guidance_scale=scale
                )

                # add noise and record each step's output latent
                noiser = DPM_Solver(model_fn, ns, predict_x0=True, thresholding=False)

                noised_sample, record_list = noiser.sample(
                    init_latent,
                    t_start=1. / model.num_timesteps,
                    t_end=encode_ratio,
                    method='multistep',
                    order=2,
                    steps=ddim_steps,
                    return_intermediate=True
                )

                # perform step wise edit
                model_fn_dst = model_wrapper(
                    lambda x, t, c: model.apply_model(x, t, c),
                    ns,
                    model_type="noise",
                    guidance_type="classifier-free",
                    condition=dst,
                    unconditional_condition=uc,
                    guidance_scale=scale
                )
                solver = DPM_Solver(model_fn_dst, ns, predict_x0=True, thresholding=False)

                solver.sample_edit = sample_edit.__get__(solver, type(solver))
                recover = solver.sample_edit(
                    noised_sample,
                    t_start=encode_ratio,
                    t_end=1. / model.num_timesteps,
                    method='multistep',
                    order=2,
                    steps=ddim_steps,
                    mask=mask,
                    record_list=list(reversed(record_list))
                )

                images = latent_to_image(model, recover)
                return images

def main():
    parser = argparse.ArgumentParser()

    parser.add_argument(
        "--prompt",
        type=str,
        nargs="?",
        default="A bowl of fruits",
        help="the prompt to render"
    )

    parser.add_argument(
        "--edit",
        type=str,
        nargs="?",
        default="A bowl of pears",
        help="the edit prompt"
    )

    parser.add_argument(
        "--ddim_steps",
        type=int,
        default=20,
        help="number of ddim sampling steps",
    )
    
    parser.add_argument(
        "--H",
        type=int,
        default=512,
        help="image height, in pixel space",
    )
    parser.add_argument(
        "--W",
        type=int,
        default=512,
        help="image width, in pixel space",
    )

    parser.add_argument(
        "--scale",
        type=float,
        default=7.5,
        help="unconditional guidance scale: eps = eps(x, empty) + scale * (eps(x, cond) - eps(x, empty))",
    )
    parser.add_argument(
        "--config",
        type=str,
        default="configs/stable-diffusion/v1-inference.yaml",
        help="path to config which constructs model",
    )
    parser.add_argument(
        "--ckpt",
        type=str,
        default="models/ldm/stable-diffusion-v1/model.ckpt",
        help="path to checkpoint of model",
    )

    parser.add_argument(
        "--seed",
        type=int,
        default=42,
        help="the seed (for reproducible sampling)",
    )
    parser.add_argument(
        "--precision",
        type=str,
        help="evaluate at this precision",
        choices=["full", "autocast"],
        default="autocast"
    )

    parser.add_argument(
        "--init_img",
        type=str,
        help="the path of image to be edit",
        default=None,
        required=True
    )
    
    parser.add_argument(
        "--outdir",
        type=str,
        help="the path of image to be edit",
        default=None,
        required=True
    )

    # opt = parser.parse_args()
    opt = parser.parse_args(args=[
        "--scale", "7.5",
        "--ddim_steps", "20",
        "--seed", "42",
        "--ckpt", "/mfs/xiangchendong19/stable-diffusion-ckpt/sd-v1-4-full-ema.ckpt",
        "--prompt", "A bowl of fruits",
        "--edit", "A bowl of pears" ,
        "--outdir", "output/",
        "--init_img", "data/fruit.png"
    ])

    config = OmegaConf.load(f"{opt.config}")
    model = load_model_from_config(config, f"{opt.ckpt}")

    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model = model.to(device)
    
    return model, opt


In [ ]:
model, opt = main()

In [ ]:
assert os.path.isfile(opt.init_img)
init_image = load_img(opt.init_img, opt).to(device)
init_image = repeat(init_image, '1 ... -> b ...', b=1)
print(opt.prompt)
print(opt.edit)
res = diffedit(model, init_image,
                src_prompt=opt.prompt,
                dst_prompt=opt.edit,
                ddim_steps=opt.ddim_steps,
                seed=opt.seed,
                scale=opt.scale,
                precision=opt.precision
                )
if not os.path.exists(opt.outdir):
    os.mkdir(opt.outdir)
Image.fromarray(res[0]).save(opt.outdir + "/diffedit.png")